# JaX Flax --> ONNX

This notebook converts brax MLP networks to an ONNX checkpoint.

In [2]:
import os

os.environ["MUJOCO_GL"] = "egl"
os.environ["PYOPENGL_PLATFORM"] = "egl"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

import jax

# Enable persistent compilation cache.
jax.config.update("jax_compilation_cache_dir", "/tmp/jax_cache")
jax.config.update("jax_persistent_cache_min_entry_size_bytes", -1)
jax.config.update("jax_persistent_cache_min_compile_time_secs", 0)

In [3]:
from brax.training.agents.ppo import networks as ppo_networks
from mujoco_playground.config import locomotion_params, manipulation_params
from mujoco_playground import locomotion, manipulation
import functools
import pickle
import jax.numpy as jp
import jax
import tf2onnx
import tensorflow as tf
from tensorflow.keras import layers
import onnxruntime as rt
from brax.training.acme import running_statistics

2025-03-12 06:05:57.684347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741759557.700928   36020 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741759557.706371   36020 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1741759557.719727   36020 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1741759557.719740   36020 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1741759557.719742   36020 computation_placer.cc:177] computation placer alr

In [16]:
from track_mjx.environment.walker import rodent
from track_mjx.agent import checkpointing
from track_mjx.agent.intention_network import Decoder
from track_mjx.analysis.rollout import create_environment
from track_mjx.environment.wrappers import EvalClipResetWrapper
from track_mjx.analysis.render import render_from_saved_rollout, display_video
from track_mjx.analysis.utils import save_to_h5py, load_from_h5py

import jax
from jax import numpy as jnp
from brax.training import distribution
from brax.training.acme import running_statistics
from pathlib import Path

# replace with your checkpoint path
ckpt_path = "/root/vast/scott-yang/track-mjx/model_checkpoints/250306_194809"
# Load config from checkpoint
ckpt = checkpointing.load_checkpoint_for_eval(ckpt_path)

cfg = ckpt["cfg"]

# make some changes to the config
# replace with absolute path to your data
# -- your notebook may not have access to the same relative path
cfg.data_path = "/root/vast/scott-yang/track-mjx/data/transform_snips.h5"
cfg.train_setup.checkpoint_to_restore = ckpt_path

# NOTE: To use accelerated JAX.JIT, only run the following code only once.
# If you re-run, you will triggers recompilations

env = create_environment(cfg)

# env_name = "BerkeleyHumanoidJoystickFlatTerrain"
# # ppo_params = locomotion_params.brax_ppo_config(env_name)
# ppo_params = locomotion_params.brax_ppo_config(env_name)z

Loading checkpoint from /root/vast/scott-yang/track-mjx/model_checkpoints/250306_194809 at step 144
env._steps_for_cur_frame: 2.0


In [5]:
obs_size = env.observation_size
act_size = env.action_size
print(obs_size, act_size)

617 38


In [74]:
cfg["network_config"]["reference_obs_size"]

470

In [18]:
# initialize the abstract decoder
network_config = cfg["network_config"]

# initialize the decoder with last layer represent the mean and variance of the action distribution
decoder = Decoder(
    network_config["decoder_layer_sizes"] + [network_config["action_size"] * 2]
)

normalizer = running_statistics.normalize
# load the normalizer parameters
normalizer_param = ckpt["policy"][0]

# load the decoder parameters
decoder_raw = ckpt["policy"][1]["params"]["decoder"]
decoder_param = {"params": decoder_raw}
# initialize the action distribution
action_distribution = distribution.NormalTanhDistribution(
    event_size=network_config["action_size"]
)

# prevent recompilation
jit_env_reset = jax.jit(env.reset, static_argnames=("clip_idx",))
jit_env_step = jax.jit(env.step)
jit_apply = jax.jit(decoder.apply)

In [55]:
normalizer_param

RunningStatisticsState(mean=Array([-2.45891337e-04,  3.16685793e-04,  4.33289912e-03, -3.88128276e-04,
        3.39214894e-04,  4.29185247e-03, -5.25006151e-04,  3.62030492e-04,
        4.25930927e-03, -6.57612400e-04,  3.85203341e-04,  4.23496263e-03,
       -7.86744349e-04,  4.08875436e-04,  4.21861047e-03,  9.90774333e-01,
       -6.58381090e-04,  1.40579010e-04, -4.37971577e-03,  9.90411460e-01,
       -6.77780539e-04,  1.03826613e-04, -6.13056868e-03,  9.89908218e-01,
       -6.95919734e-04,  6.94825139e-05, -7.87717476e-03,  9.89281356e-01,
       -7.12124922e-04,  3.75052223e-05, -9.61835682e-03,  9.88553762e-01,
       -7.25791324e-04,  8.07309061e-06, -1.13530448e-02,  5.11902608e-02,
       -2.24598959e-01, -9.97642707e-03, -1.45080999e-01, -7.18526393e-02,
       -2.88108051e-01, -4.12455015e-02, -1.95677921e-01, -6.73351139e-02,
       -1.52951419e-01, -1.06078126e-01, -2.88545609e-01, -7.89830834e-03,
        2.42688763e-03, -2.86441529e-03,  4.61474657e-02,  2.33872104e-0

In [104]:
from tensorflow.keras import layers


class DecoderTF(tf.keras.Model):
    def __init__(
        self,
        layer_sizes,
        activation=tf.nn.relu,
        kernel_init="lecun_uniform",
        activate_final=False,
        bias=True,
        mean_std=None,
    ):
        """
        Initializes the Decoder, including the observation normalizer

        Args:
            layer_sizes (Sequence[int]): List of layer sizes for each Dense layer.
            activation (callable): Activation function to apply.
            kernel_init (str or callable): Kernel initializer for Dense layers.
            activate_final (bool): Whether to apply activation (and layer norm) on the final layer.
            bias (bool): Whether the Dense layers should use a bias term.
            mean_std (tuple): Mean and standard deviation for obs normalization.
        """
        super().__init__()
        self.layer_sizes = layer_sizes
        self.activation = activation
        self.kernel_init = kernel_init
        self.activate_final = activate_final
        self.bias = bias
        if mean_std is not None:
            self.mean = tf.Variable(mean_std[0], trainable=False, dtype=tf.float32)
            self.std = tf.Variable(mean_std[1], trainable=False, dtype=tf.float32)
        else:
            self.mean = None
            self.std = None

        # Build lists to store Dense layers and corresponding LayerNorm layers.
        self.dense_layers = []
        self.layer_norms = []

        for i, size in enumerate(self.layer_sizes):
            dense_layer = layers.Dense(
                size,
                kernel_initializer=self.kernel_init,
                use_bias=self.bias,
                name=f"hidden_{i}",
            )
            self.dense_layers.append(dense_layer)
            # Apply activation and layer norm if it's not the final layer or if activate_final is True.
            if i != len(self.layer_sizes) - 1 or self.activate_final:
                self.layer_norms.append(
                    layers.LayerNormalization(name=f"LayerNorm_{i}")
                )
            else:
                self.layer_norms.append(None)

    def call(self, inputs, get_activation=False):
        """
        Forward pass through the network.

        Args:
            inputs (tf.Tensor): Input tensor.
            get_activation (bool): If True, also return a dict with activations per layer.

        Returns:
            tf.Tensor or Tuple[tf.Tensor, dict]: The output tensor, and optionally a dictionary
            mapping layer names to their activations.
        """
        if isinstance(inputs, list):
            inputs = inputs[0]
        activations = {}
        if self.mean is not None and self.std is not None:
            # Normalize the part of the input starting at column 60 (intention size TODO: config this).
            normalized_part = (inputs[:, 60:] - self.mean) / self.std
            # Concatenate the unchanged part with the normalized part along axis 1.
            inputs = tf.concat([inputs[:, :60], normalized_part], axis=1)
        x = inputs
        for i, dense_layer in enumerate(self.dense_layers):
            x = dense_layer(x)
            # Apply activation (and layer norm) for all layers except the final one
            # unless activate_final is True.
            if i != len(self.layer_sizes) - 1 or self.activate_final:
                x = self.activation(x)
                if self.layer_norms[i] is not None:
                    x = self.layer_norms[i](x)
                if get_activation:
                    activations[f"layer_{i}"] = x
        if get_activation:
            return x, activations
        return x


def make_policy_network(
    param_size,
    hidden_layer_sizes=[512, 512, 512],
    mean_std=None,
):
    policy_network = DecoderTF(
        layer_sizes=list(hidden_layer_sizes) + [param_size],
        mean_std=mean_std,
    )
    return policy_network

In [105]:
ref_obs_size = cfg["network_config"]["reference_obs_size"]
mean_std = (normalizer_param.mean[ref_obs_size:], normalizer_param.std[ref_obs_size:])
tf_policy_network = make_policy_network(
    param_size=act_size * 2,
    mean_std=mean_std,
)

In [106]:
example_input = tf.zeros((1, 60 + 147))
example_output = tf_policy_network(example_input)
print(example_output.shape)

(1, 76)


In [107]:
tf_policy_network.layers

[<Dense name=hidden_0, built=True>,
 <LayerNormalization name=LayerNorm_0, built=True>,
 <Dense name=hidden_1, built=True>,
 <LayerNormalization name=LayerNorm_1, built=True>,
 <Dense name=hidden_2, built=True>,
 <LayerNormalization name=LayerNorm_2, built=True>,
 <Dense name=hidden_3, built=True>]

In [108]:
import numpy as np
import tensorflow as tf


def transfer_weights(jax_params, tf_model):
    """
    Transfer weights from a JAX parameter dictionary to the TensorFlow model.

    Parameters:
    - jax_params: dict
      Nested dictionary with structure {block_name: {layer_name: {params}}}.
      For example:
      {
        'CNN_0': {
          'Conv_0': {'kernel': np.ndarray},
          'Conv_1': {'kernel': np.ndarray},
          'Conv_2': {'kernel': np.ndarray},
        },
        'MLP_0': {
          'hidden_0': {'kernel': np.ndarray, 'bias': np.ndarray},
          'hidden_1': {'kernel': np.ndarray, 'bias': np.ndarray},
          'hidden_2': {'kernel': np.ndarray, 'bias': np.ndarray},
        }
      }

    - tf_model: tf.keras.Model
      An instance of the adapted VisionMLP model containing named submodules and layers.
    """
    for layer_name, layer_params in jax_params.items():
        try:
            tf_layer = tf_model.get_layer(name=layer_name)
        except ValueError:
            print(f"Layer {layer_name} not found in TensorFlow model.")
            continue
        if isinstance(tf_layer, tf.keras.layers.Dense):
            kernel = np.array(layer_params["kernel"])
            bias = np.array(layer_params["bias"])
            print(
                f"Transferring Dense layer {layer_name}, kernel shape {kernel.shape}, bias shape {bias.shape}"
            )
            tf_layer.set_weights([kernel, bias])
        elif isinstance(tf_layer, tf.keras.layers.LayerNormalization):
            gamma = np.array(layer_params["scale"])
            beta = np.array(layer_params["bias"])
            print(
                f"Transferring LayerNorm layer {layer_name}, gamma shape {gamma.shape}, beta shape {beta.shape}"
            )
            tf_layer.set_weights([gamma, beta])
        else:
            print(f"Unhandled layer type in {layer_name}: {type(tf_layer)}")

    print("Weights transferred successfully.")

In [109]:
transfer_weights(decoder_raw, tf_policy_network)

Transferring LayerNorm layer LayerNorm_0, gamma shape (512,), beta shape (512,)
Transferring LayerNorm layer LayerNorm_1, gamma shape (512,), beta shape (512,)
Transferring LayerNorm layer LayerNorm_2, gamma shape (512,), beta shape (512,)
Transferring Dense layer hidden_0, kernel shape (207, 512), bias shape (512,)
Transferring Dense layer hidden_1, kernel shape (512, 512), bias shape (512,)
Transferring Dense layer hidden_2, kernel shape (512, 512), bias shape (512,)
Transferring Dense layer hidden_3, kernel shape (512, 76), bias shape (76,)
Weights transferred successfully.


In [110]:
output_path = "decoder.onnx"

# Example inputs for the model
test_input = [np.ones((1, 60 + 147), dtype=np.float32)]

# Define the TensorFlow input signature
spec = [tf.TensorSpec(shape=(1, 60 + 147), dtype=tf.float32, name="obs")]

tensorflow_pred = tf_policy_network(test_input)[0]
# Build the model by calling it with example data
print(f"Tensorflow prediction: {tensorflow_pred}")

tf_policy_network.output_names = ["continuous_actions"]

# opset 11 matches isaac lab.
model_proto, _ = tf2onnx.convert.from_keras(
    tf_policy_network, input_signature=spec, opset=11, output_path=output_path
)

# Run inference with ONNX Runtime
output_names = ["continuous_actions"]
providers = ["CPUExecutionProvider"]
m = rt.InferenceSession(output_path, providers=providers)

Tensorflow prediction: [ 4.1564755  -1.0937533  -2.318982    0.23866072 -2.4380524   0.13061154
 -1.6350007   2.434084    1.0309069  -1.7794216   3.042029    1.0608172
 -0.46156973 -1.361636    2.749161    1.7966634   0.11604372  5.595267
 -3.001744    0.03239346 -0.05042651  0.8248224  -1.9539303   0.373685
 -0.5999829  -3.7575974   2.578617    1.8515337  -2.8840134  -1.1817415
 -0.5334719  -2.1998577   2.570844    0.29747787  0.20663525  1.1557158
 -2.5952613   0.9801689  -2.9540143  -1.9215935  -6.597722   -3.8621898
 -3.7796564  -2.2326736  -4.091821   -5.1267385  -2.6486428  -1.641351
 -5.689634   -4.79763    -6.607268   -3.2419324  -3.725782   -3.228977
 -5.44276    -9.1216545  -5.4976506  -3.4675915  -5.5660768  -1.3366315
 -1.1810958  -4.170766   -2.3867505  -2.0194445  -4.1066194  -2.707123
 -3.1345754  -2.3324502  -3.859407   -4.0721936  -3.8622735  -4.599808
 -3.1127875  -6.624743   -4.282755   -4.013839  ]


I0000 00:00:1741764286.924626   36020 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
I0000 00:00:1741764286.924859   36020 single_machine.cc:374] Starting new session
I0000 00:00:1741764286.931367   36020 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 42374 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:17:00.0, compute capability: 8.6
I0000 00:00:1741764287.375772   36020 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 42374 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:17:00.0, compute capability: 8.6
I0000 00:00:1741764287.591420   36020 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
I0000 00:00:1741764287.591617   36020 single_machine.cc:374] Starting new session
I0000 00:00:1741764287.597933   36020 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 42374 MB memory: 

In [114]:
onnx_input = {"obs": np.ones((1, 147 + 60), dtype=np.float32)}
# Prepare inputs for ONNX Runtime
onnx_pred = m.run(output_names, onnx_input)[0][0]


print("ONNX prediction shape:", onnx_pred.shape)
print("ONNX prediction:", onnx_pred)

ONNX prediction shape: (76,)
ONNX prediction: [ 4.156474   -1.0937538  -2.318984    0.23865947 -2.4380517   0.13061145
 -1.635001    2.4340837   1.0309054  -1.7794225   3.0420287   1.0608153
 -0.46156836 -1.3616365   2.7491605   1.7966647   0.11604482  5.595266
 -3.0017443   0.03239317 -0.05042624  0.8248226  -1.9539304   0.3736849
 -0.59998286 -3.7575984   2.5786164   1.8515334  -2.8840137  -1.1817414
 -0.5334718  -2.1998582   2.5708437   0.2974775   0.20663527  1.1557169
 -2.5952616   0.98016953 -2.9540124  -1.9215932  -6.597722   -3.8621893
 -3.779657   -2.2326732  -4.0918202  -5.1267385  -2.6486433  -1.6413513
 -5.6896324  -4.7976274  -6.607269   -3.2419322  -3.725782   -3.2289767
 -5.4427605  -9.121656   -5.4976516  -3.4675908  -5.566077   -1.3366319
 -1.1810962  -4.170768   -2.386751   -2.0194447  -4.106618   -2.7071228
 -3.1345768  -2.3324504  -3.8594072  -4.0721936  -3.8622742  -4.5998077
 -3.1127877  -6.6247416  -4.282754   -4.013839  ]


In [ ]:
test_input = {
    "state": jp.ones(obs_size["state"]),
    "privileged_state": jp.zeros(obs_size["privileged_state"]),
}
jax_pred, _ = inference_fn(test_input, jax.random.PRNGKey(0))
print(jax_pred)

In [ ]:
import matplotlib.pyplot as plt

print(onnx_pred.shape)
print(tensorflow_pred.shape)
print(jax_pred.shape)
plt.plot(onnx_pred, label="onnx")
plt.plot(tensorflow_pred, label="tensorflow")
plt.plot(jax_pred, label="jax")
plt.legend()
plt.show()

(76,)
(76,)


NameError: name 'jax_pred' is not defined

In [20]:
# env_cfg = locomotion.get_default_config(env_name)
# env = locomotion.load(env_name, config=env_cfg)
# jit_reset = jax.jit(env.reset)
# jit_step = jax.jit(env.step)

In [21]:
# # Test the policy.

# # env_cfg = locomotion.get_default_config(env_name)
# # env_cfg.init_from_crouch = 0.0
# # env = locomotion.load(env_name, config=env_cfg)
# # env_cfg = manipulation.get_default_config(env_name)
# # env = manipulation.load(env_name, config=env_cfg)
# # jit_reset = jax.jit(env.reset)
# # jit_step = jax.jit(env.step)

# x = 0.8
# y = 0.0
# yaw = 0.3
# command = jp.array([x, y, yaw])
# # actions = []

# states = [state := jit_reset(jax.random.PRNGKey(555))]
# state.info["command"] = command
# for _ in range(env_cfg.episode_length):
#   onnx_input = {'obs': np.array(state.obs["state"].reshape(1, -1))}
#   action = m.run(output_names, onnx_input)[0][0]
#   state = jit_step(state, jp.array(action))
#   state.info["command"] = command
#   states.append(state)
#   # actions.append(state.info["motor_targets"])
#   # actions.append(action)
#   if state.done:
#     print("Unexpected termination.")
#     break

In [22]:
# import mediapy as media
# fps = 1.0 / env.dt

# frames = env.render(
#     states,
#     camera="track",
#     width=640*2,
#     height=480*2,
# )
# media.show_video(frames, fps=fps, loop=False)